This module will be similar to SQL
1. How to merge DataFrames
2. How to concatenate DataFrames
3. Identify unique values in one DF but not in other DF
4. Identify values shared in both DFs
5. Typical joining operation

In [1]:
import pandas as pd

In [2]:
week1 = pd.read_csv('resources/Restaurant - Week 1 Sales.csv')
week2 = pd.read_csv('resources/Restaurant - Week 2 Sales.csv')
customers = pd.read_csv('resources/Restaurant - Customers.csv')
foods = pd.read_csv('resources/Restaurant - Foods.csv')

<h3>The pd.concat() Method</h3>

In [3]:
len(week1)

250

In [4]:
len(week2)

250

In [5]:
pd.concat([week1,week2], ignore_index=True).tail() #Pandas keeps index from both DFs by default. ignore_index=False

,Customer ID,Food ID
495,783,10
496,556,10
497,547,9
498,252,9
499,249,6


In [6]:
sales = pd.concat([week1,week2], keys=['week 1', 'week 2']) # Multi layer indexed DF

In [7]:
sales.loc['week 2'].tail()

,Customer ID,Food ID
245,783,10
246,556,10
247,547,9
248,252,9
249,249,6


In [8]:
sales.loc[('week 2', 240), 'Customer ID']

734

<h3>The .append() Method on a DataFrame</h3>

In [9]:
week1 = pd.read_csv('resources/Restaurant - Week 1 Sales.csv')
week2 = pd.read_csv('resources/Restaurant - Week 2 Sales.csv')
customers = pd.read_csv('resources/Restaurant - Customers.csv')
foods = pd.read_csv('resources/Restaurant - Foods.csv')

In [10]:
sales = week2.append(week1, ignore_index=True)

<h3>Inner Joins</h3>
<a href='http://i.stack.imgur.com/hzl8e.png' target="_blank">Refer</a><br><br>
On single column<br>
Data Common in both DFs

In [11]:
week1.merge(week2, how='inner', on='Customer ID', suffixes=[' - week 1',' - week 2']).head(3)

,Customer ID,Food ID - week 1,Food ID - week 2
0,537,9,5
1,155,9,3
2,155,1,3


In [12]:
week1[week1['Customer ID'] == 155]

,Customer ID,Food ID
4,155,9
17,155,1


In [13]:
week2[week2['Customer ID'] == 155]

,Customer ID,Food ID
208,155,3


On multiple columns

In [14]:
week1 = pd.read_csv('resources/Restaurant - Week 1 Sales.csv')
week2 = pd.read_csv('resources/Restaurant - Week 2 Sales.csv')
customers = pd.read_csv('resources/Restaurant - Customers.csv')
foods = pd.read_csv('resources/Restaurant - Foods.csv')

In [15]:
week1.merge(week2, how='inner', on=['Customer ID', 'Food ID']) # Same customer , same food on both weeks

,Customer ID,Food ID
0,304,3
1,540,3
2,937,10
3,233,3
4,21,4
5,21,4
6,922,1
7,578,5
8,578,5


In [16]:
week1[week1['Customer ID']==21]

,Customer ID,Food ID
101,21,4
212,21,4


In [17]:
week2[week2['Customer ID']==21]

,Customer ID,Food ID
30,21,4


In [18]:
week1[week1['Customer ID']==578]

,Customer ID,Food ID
224,578,5


In [19]:
week2[week2['Customer ID']==578]

,Customer ID,Food ID
29,578,5
189,578,5


<h3>Outer Joins</h3>
<a href='http://i.stack.imgur.com/hzl8e.png' target="_blank">Refer</a><br><br>
Data found in either DFs and both together -  this is pandas default behaviour<br>
Data found in either DFs but not both

In [20]:
week1 = pd.read_csv('resources/Restaurant - Week 1 Sales.csv')
week2 = pd.read_csv('resources/Restaurant - Week 2 Sales.csv')
customers = pd.read_csv('resources/Restaurant - Customers.csv')
foods = pd.read_csv('resources/Restaurant - Foods.csv')

In [25]:
outer_join = week1.merge(week2, how='outer', on ='Customer ID', suffixes=[' - week 1' , ' - week 2'], indicator=True)
#Default behaviour, include data from left, right and common,
# Note : this is different than simple concatenating

In [24]:
len(outer_join)

454

In [26]:
outer_join

,Customer ID,Food ID - week 1,Food ID - week 2,_merge
0,537,9.0,5.0,both
1,97,4.0,NaN,left_only
2,658,1.0,NaN,left_only
3,202,2.0,NaN,left_only
4,155,9.0,3.0,both
...,...,...,...,...
449,855,NaN,4.0,right_only
450,559,NaN,10.0,right_only
451,276,NaN,4.0,right_only
452,556,NaN,10.0,right_only


In [27]:
outer_join['_merge'].value_counts()

right_only    197
left_only     195
both           62
Name: _merge, dtype: int64

In [29]:
mask = outer_join['_merge'].isin(['left_only','right_only']) # We cut out the both
outer_join[mask]

,Customer ID,Food ID - week 1,Food ID - week 2,_merge
1,97,4.0,NaN,left_only
2,658,1.0,NaN,left_only
3,202,2.0,NaN,left_only
6,213,8.0,NaN,left_only
7,600,1.0,NaN,left_only
...,...,...,...,...
449,855,NaN,4.0,right_only
450,559,NaN,10.0,right_only
451,276,NaN,4.0,right_only
452,556,NaN,10.0,right_only


<h3>Left Joins</h3>
<a href='http://i.stack.imgur.com/hzl8e.png' target="_blank">Refer</a><br><br>
Right Join is just opposit <br> Instead use left join with flipped data sets

In [31]:
week1 = pd.read_csv('resources/Restaurant - Week 1 Sales.csv')
week2 = pd.read_csv('resources/Restaurant - Week 2 Sales.csv')
customers = pd.read_csv('resources/Restaurant - Customers.csv')
foods = pd.read_csv('resources/Restaurant - Foods.csv')

In [32]:
week1.head(3)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1


In [33]:
foods.head(3)

,Food ID,Food Item,Price
0,1,Sushi,3.99
1,2,Burrito,9.99
2,3,Taco,2.99


In [37]:
left_join = week1.merge(foods, how='left', on='Food ID', sort=True) 
# sort on common column, i.e, 'Food ID'

In [38]:
left_join.head()

,Customer ID,Food ID,Food Item,Price
0,658,1,Sushi,3.99
1,600,1,Sushi,3.99
2,155,1,Sushi,3.99
3,341,1,Sushi,3.99
4,20,1,Sushi,3.99


<h3>The left_on and right_on Parameters</h3>

In [39]:
week1 = pd.read_csv('resources/Restaurant - Week 1 Sales.csv')
week2 = pd.read_csv('resources/Restaurant - Week 2 Sales.csv')
customers = pd.read_csv('resources/Restaurant - Customers.csv')
foods = pd.read_csv('resources/Restaurant - Foods.csv')

In [40]:
week2.head(3)

,Customer ID,Food ID
0,688,10
1,813,7
2,495,10


In [41]:
customers.head(3)

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer
2,3,Roger,Black,Male,Tagfeed,Account Executive


NOTE : Key name is different in week2 ('Customer ID') and customers ('ID') DFs

In [44]:
week2.merge(customers, how='left', left_on='Customer ID', right_on='ID', sort=True).drop('ID', axis=1).head(10)

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation
0,8,6,Frances,Adams,Female,Dabshots,Developer III
1,13,2,Ruth,Alvarez,Female,Twitterlist,Mechanical Systems Engineer
2,21,4,Albert,Burns,Male,Rhynoodle,Junior Executive
3,24,8,Donna,Thomas,Female,Jaxbean,Chief Design Engineer
4,27,4,Jessica,Bennett,Female,Twitternation,Account Executive
5,30,4,Pamela,Hicks,Female,Ntags,Cost Accountant
6,35,8,Nicole,Chapman,Female,Pixonyx,Electrical Engineer
7,39,10,Teresa,Harrison,Female,Oba,Payment Adjustment Coordinator
8,45,8,Earl,Marshall,Male,Agivu,Dental Hygienist
9,46,6,Adam,Cole,Male,Trupe,Accountant III


<h3>Merging by Indexes with the left_index and right_index Parameters</h3>

In [45]:
week1 = pd.read_csv('resources/Restaurant - Week 1 Sales.csv')
week2 = pd.read_csv('resources/Restaurant - Week 2 Sales.csv')
customers = pd.read_csv('resources/Restaurant - Customers.csv', index_col='ID')
foods = pd.read_csv('resources/Restaurant - Foods.csv', index_col='Food ID')

In [46]:
customers.head(2)

,First Name,Last Name,Gender,Company,Occupation
ID,,,,,
1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer


In [48]:
foods.head(2)

,Food Item,Price
Food ID,,
1,Sushi,3.99
2,Burrito,9.99


In [49]:
week1.head(2)

,Customer ID,Food ID
0,537,9
1,97,4


In [51]:
sales = week1.merge(customers, how='left', left_on='Customer ID', right_index=True)
sales.head()

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation
0,537,9,Cheryl,Carroll,Female,Zoombeat,Registered Nurse
1,97,4,Amanda,Watkins,Female,Ozu,Account Coordinator
2,658,1,Patrick,Webb,Male,Browsebug,Community Outreach Specialist
3,202,2,Louis,Campbell,Male,Rhynoodle,Account Representative III
4,155,9,Carolyn,Diaz,Female,Gigazoom,Database Administrator III


In [53]:
sales = sales.merge(foods, how='left', left_on='Food ID', right_index=True)
sales.head()

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation,Food Item,Price
0,537,9,Cheryl,Carroll,Female,Zoombeat,Registered Nurse,Donut,0.99
1,97,4,Amanda,Watkins,Female,Ozu,Account Coordinator,Quesadilla,4.25
2,658,1,Patrick,Webb,Male,Browsebug,Community Outreach Specialist,Sushi,3.99
3,202,2,Louis,Campbell,Male,Rhynoodle,Account Representative III,Burrito,9.99
4,155,9,Carolyn,Diaz,Female,Gigazoom,Database Administrator III,Donut,0.99


In [54]:
week1.head(2)

,Customer ID,Food ID
0,537,9
1,97,4


In [55]:
week2.head(2)

,Customer ID,Food ID
0,688,10
1,813,7


In [57]:
week1.merge(week2, how='left', left_index=True, right_index=True, suffixes=[' - week 1', ' - week 2'])

,Customer ID - week 1,Food ID - week 1,Customer ID - week 2,Food ID - week 2
0,537,9,688,10
1,97,4,813,7
2,658,1,495,10
3,202,2,189,5
4,155,9,267,3
...,...,...,...,...
245,413,9,783,10
246,926,6,556,10
247,134,3,547,9
248,396,6,252,9


<h3>The .join() Method</h3>

In [60]:
week1 = pd.read_csv('resources/Restaurant - Week 1 Sales.csv')
week2 = pd.read_csv('resources/Restaurant - Week 2 Sales.csv')
customers = pd.read_csv('resources/Restaurant - Customers.csv')
foods = pd.read_csv('resources/Restaurant - Foods.csv')
satisfaction = pd.read_csv('resources/Restaurant - Week 1 Satisfaction.csv')

In [61]:
week1.head(2)

,Customer ID,Food ID
0,537,9
1,97,4


In [62]:
satisfaction.head(2)

,Satisfaction Rating
0,2
1,7


In [67]:
# brute force
week1.merge(satisfaction, how='left', left_index=True, right_index=True).head()

,Customer ID,Food ID,Satisfaction Rating
0,537,9,2
1,97,4,7
2,658,1,3
3,202,2,7
4,155,9,10


In [66]:
# less code
week1.join(satisfaction).head()

,Customer ID,Food ID,Satisfaction Rating
0,537,9,2
1,97,4,7
2,658,1,3
3,202,2,7
4,155,9,10


<h3>The pd.merge() Method</h3>
Just another way of doing same above things

In [68]:
week1.head(2)

,Customer ID,Food ID
0,537,9
1,97,4


In [69]:
customers.head(2)

,ID,First Name,Last Name,Gender,Company,Occupation
0,1,Joseph,Perkins,Male,Dynazzy,Community Outreach Specialist
1,2,Jennifer,Alvarez,Female,DabZ,Senior Quality Engineer


In [71]:
pd.merge(week1, customers, how='left', left_on='Customer ID', right_on='ID').head()

,Customer ID,Food ID,ID,First Name,Last Name,Gender,Company,Occupation
0,537,9,537,Cheryl,Carroll,Female,Zoombeat,Registered Nurse
1,97,4,97,Amanda,Watkins,Female,Ozu,Account Coordinator
2,658,1,658,Patrick,Webb,Male,Browsebug,Community Outreach Specialist
3,202,2,202,Louis,Campbell,Male,Rhynoodle,Account Representative III
4,155,9,155,Carolyn,Diaz,Female,Gigazoom,Database Administrator III
